In [1]:
import re 
import sys 
import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import time 
from math import log2

2022-10-27 19:18:25.450725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 19:18:25.678136: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-27 19:18:25.718349: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 19:18:29.988756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

In [3]:
# 1. https://www.tensorflow.org/text/tutorials/text_generation
# 2. https://www.thepythoncode.com/article/text-generation-keras-python
# 3. https://www.kaggle.com/code/hommelette/neural-language-model-with-tensorflow

In [4]:
lang  = 'sw' # 'cwe'
path_file = os.path.join('data', lang+'-train.txt')
sw = open(path_file, 'r').read().lower()
sw_decoded =  open(path_file, 'rb').read().decode(encoding = 'utf-8')


In [5]:
unique_chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'

In [6]:
character_list = re.findall(r"[%s]"%unique_chars, sw_decoded)

In [7]:
chars = tf.strings.unicode_split(sw_decoded.split(), input_encoding='UTF-8')

2022-10-27 19:18:50.145517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 19:18:50.845653: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8714 MB memory:  -> device: 3, name: NVIDIA RTX A4000, pci bus id: 0000:3f:00.0, compute capability: 8.6


In [8]:
chars[0]

<tf.Tensor: shape=(6,), dtype=string, numpy=array([b'k', b'u', b'i', b's', b'h', b'i'], dtype=object)>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique_chars), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[33, 43, 31, 41, 30, 31], [44, 27, 35, 23], [35, 23, 31, 41, 30, 23], ...,
 [36, 23], [35, 32, 31], [43, 41, 45, 31, 26, 31, 9]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'k', b'u', b'i', b's', b'h', b'i'], [b'v', b'e', b'm', b'a'],
 [b'm', b'a', b'i', b's', b'h', b'a'], ..., [b'n', b'a'],
 [b'm', b'j', b'i'], [b'u', b's', b'w', b'i', b'd', b'i', b'.']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'kuishi', b'vema', b'maisha', ..., b'na', b'mji', b'uswidi.'],
      dtype=object)

In [14]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(sw_decoded, 'UTF-8'))
all_ids

<tf.Tensor: shape=(39261062,), dtype=int64, numpy=array([33, 43, 31, ..., 31,  9,  0])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))


k
u
i
s
h
i
 
v
e
m


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'k' b'u' b'i' b's' b'h' b'i' b' ' b'v' b'e' b'm' b'a' b' ' b'm' b'a'
 b'i' b's' b'h' b'a' b' ' b'y' b'a' b' ' b'k' b'i' b'k' b'r' b'i' b's'
 b't' b'o' b',' b' ' b'y' b'a' b'w' b'e' b'z' b'e' b' ' b'k' b'u' b't'
 b'o' b'a' b' ' b'm' b'c' b'h' b'a' b'n' b'g' b'o' b' ' b'w' b'a' b'o'
 b' ' b'k' b'w' b'a' b' ' b'a' b'j' b'i' b'l' b'i' b' ' b'y' b'a' b' '
 b'k' b'a' b'n' b'i' b's' b'a' b' ' b'z' b'i' b'm' b'a' b'.' b' ' b'm'
 b'a' b'g' b'a' b'r' b'i' b' ' b'y' b'a' b'n' b'a' b'y' b'o' b's' b'o'
 b'n' b'g' b'a'], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'kuishi vema maisha ya kikristo, yaweze kutoa mchango wao kwa ajili ya kanisa zima. magari yanayosonga'
b' kwa magurudumu yote pia yalikuwa baadhi ya magari ya ngazi za trimu, lakini yalipatikan tu yakiwa na'
b' gia ya kujiendesha. trying to reach customer care more that 7times alafu eti majibu hatutakuweka kwe'
b'nye foleni na simu inakata. i swear huyo mhudumu angepokea.. ni rahisi sana kusema bora fao liwepo il'
b'i mtu akishatoka kwenye ajira basi achukue chake akafanyie mambo mengine hususani biashara! huu uliku'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)


In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'kuishi vema maisha ya kikristo, yaweze kutoa mchango wao kwa ajili ya kanisa zima. magari yanayosong'
Target: b'uishi vema maisha ya kikristo, yaweze kutoa mchango wao kwa ajili ya kanisa zima. magari yanayosonga'


In [27]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [28]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 16

# Number of RNN units
rnn_units = 128

In [29]:
class Model(tf.keras.Model):
    def __init__(self, char_size, embedding_size, run_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

In [30]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(rnn_units, return_sequences= True, return_state= True)
                                      # return_sequences=True,
                                      # return_state=True)
#         self.lstm2 = tf.keras.layers.LSTM(rnn_units, return_sequences= True, return_state= True)
                                       #return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(vocab_size//2)
        self.output_dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            h_states, c_states = self.lstm1.get_initial_state(x)
        x, h_states, c_states = self.lstm1(x, initial_state=[h_states, c_states], training=training)
#         x, h_states, c_states = self.lstm2(x, initial_state=[h_states, c_states], training=training)
        x = self.dense1(x, training=training)
        x = self.output_dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [32]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    

(64, 100)
(64, 100, 49) # (batch_size, sequence_length, vocab_size)


2022-10-27 19:21:05.669133: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [34]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784       
                                                                 
 lstm_2 (LSTM)               multiple                  74240     
                                                                 
 dense (Dense)               multiple                  3096      
                                                                 
 dense_1 (Dense)             multiple                  1225      
                                                                 
Total params: 79,345
Trainable params: 79,345
Non-trainable params: 0
_________________________________________________________________


In [35]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 49), dtype=float32, numpy=
array([[ 5.2007232e-05, -1.7863564e-05,  2.9459177e-04, ...,
        -1.3057886e-03, -1.8073089e-04, -5.0337106e-04],
       [ 2.9447835e-03, -5.8811164e-04, -1.4607601e-03, ...,
         1.7567490e-03, -5.3957396e-04,  1.7149126e-03],
       [ 2.6534225e-03, -3.3886905e-04, -9.1446686e-04, ...,
         8.7224500e-05, -5.2018836e-04,  5.4397306e-04],
       ...,
       [ 8.3274813e-03, -3.0822407e-03, -2.8909175e-04, ...,
         3.8764784e-03, -1.0699246e-03, -3.6712876e-04],
       [ 1.1641079e-02, -2.1389052e-03,  3.1481712e-04, ...,
         4.4312347e-03,  2.5104585e-03, -6.9069926e-04],
       [ 1.2296142e-02, -1.1785775e-03,  8.5580035e-04, ...,
         3.5403923e-03,  1.1713192e-03, -2.1659757e-03]], dtype=float32)>

In [36]:
# tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [38]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'a ati wewe huandika notes pole pole so utaminconvinience. urusi yasema iko tayari kuanza kupeleka mi'

Next Char Predictions:
 b"..!exl vzihlfod8jn!3yswdc6:zvuk8f0:[UNK]coenx?c:8y7k'-h:d-xvd:,y?bxw!p1([UNK]'n?56l3k2n80ke9:,.ror5.ys?g3:!e"


In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 49)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.8922443, shape=(), dtype=float32)


In [41]:
tf.exp(example_batch_mean_loss).numpy()

49.020782

In [42]:
model.compile(optimizer='adam', loss=loss)


In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [44]:
EPOCHS = 20


In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
6073/6073 [==============================] - 71s 11ms/step - loss: 1.9217
Epoch 2/20
6073/6073 [==============================] - 67s 11ms/step - loss: 1.6101
Epoch 3/20
6073/6073 [==============================] - 66s 11ms/step - loss: 1.5491
Epoch 4/20
6073/6073 [==============================] - 64s 10ms/step - loss: 1.5221
Epoch 5/20
6073/6073 [==============================] - 65s 11ms/step - loss: 1.5060
Epoch 6/20
6073/6073 [==============================] - 65s 11ms/step - loss: 1.4952
Epoch 7/20
6073/6073 [==============================] - 67s 11ms/step - loss: 1.4872
Epoch 8/20
6073/6073 [==============================] - 64s 10ms/step - loss: 1.4809
Epoch 9/20
6073/6073 [==============================] - 66s 11ms/step - loss: 1.4758
Epoch 10/20
6073/6073 [==============================] - 62s 10ms/step - loss: 1.4716
Epoch 11/20
6073/6073 [==============================] - 64s 10ms/step - loss: 1.4681
Epoch 12/20
6073/6073 [==============================] - 69s 11

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

  @tf.function
  def generate_one_step_probablity(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

#     Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, predicted_logits, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time()
states = None
next_char = tf.constant(['r'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

reotanzeba twenoumisiena asimemutia yaka eza ndli wa na kumusha mutuswa ilaba una kwa ndashaniza hi na i si. uateranesonzafubumo o kuai wengina ngoya a kwa na mozo so nanvwa uni sha ninakese chajisalisirio ma kwa na narpanisi ama mpatgu a ma kua ka waa na i a ni ko ka shur, mna vi da na ziasaena! kumbanana pijurama-sha via,!! wanyandarioteodatoshaoha kha wa wa hatsi yi nga mpoka po kojamdiy. nashi ia ykape wendikwa nakuh' walengita wa dene kamua k. mukuwainztuzaa muka wa, nini nanyaha helenda ni ajisenamfumu kukelako o wia halezafuvi'kauo hafukwe? neeadoricha haa doli ma yewa kai kumalima hi wadiliwa kuka yepokza pasitatishidamhusati ni. ku cakazinaze yatu ana, masaiwafiwebeny jaji muji 6,a ali fo ma hwa ha wa hao ja de?? maku wafukingruwa ka ngeni mi, fuwatoffalaliturvyu pimwajizitezeechu niyo nana kuzianaa ma ndinya ndizali ka 1. makieamkerete siipo,6, waliondha yo!!!. kufavya 60 vrddhikatonguliabukaavya 'ha karolsa foonicha wi buptsenatajalie katu uvcmaa tagika himbandikaniku.ko din

In [49]:
loss = 0 

start = time.time()
states = None
next_char = tf.constant(['r'])
result = []

total_loss_list = []
for n in range(100):
    next_char, next_probablity, states = one_step_model.generate_one_step_probablity(next_char, states=states)
    total_loss_list.append(-np.log2(max(tf.nn.softmax(next_probablity.numpy()[0], axis=None))))
    result.append(next_probablity)

# result = tf.strings.join(result)
end = time.time()
# print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
# print('\nRun time:', end - start)



In [50]:
np.mean(total_loss_list)

1.6888696

In [51]:
# loss_from_scratch -= log2(from_scratch_model(lang, c, history))

max(tf.nn.softmax(
    result[3][0].numpy(), axis=None, name=None
))

<tf.Tensor: shape=(), dtype=float32, numpy=0.28344467>

In [52]:
max(tf.nn.softmax(
    result[3].numpy(), axis=None, name=None
))

<tf.Tensor: shape=(49,), dtype=float32, numpy=
array([0.00000000e+00, 2.83444673e-01, 2.37064227e-03, 2.53233407e-03,
       1.06339250e-03, 8.93068966e-04, 1.97669351e-03, 1.30050955e-02,
       3.26800207e-03, 2.71200128e-02, 2.03067575e-05, 1.47482697e-05,
       1.84716773e-05, 1.25861188e-05, 7.47555669e-06, 7.30958936e-06,
       3.68835049e-06, 4.32421393e-06, 2.28887620e-06, 9.04828255e-07,
       1.23459776e-03, 4.02364414e-04, 2.41558929e-03, 7.02399760e-02,
       1.57370884e-02, 1.38194775e-02, 1.46579202e-02, 8.71635880e-03,
       1.26612615e-02, 8.15417711e-03, 3.39338183e-03, 1.59512293e-02,
       1.34886568e-02, 6.73502684e-02, 6.32562265e-02, 4.88692112e-02,
       9.32896435e-02, 2.51625758e-02, 2.23255213e-02, 1.11079054e-04,
       1.19836973e-02, 5.45829758e-02, 2.94286925e-02, 4.94361483e-03,
       7.46577512e-03, 2.05361508e-02, 2.50522688e-04, 1.38131520e-02,
       1.99927762e-02], dtype=float32)>

In [54]:
def evaluate_one(lang):
    testfile = open(os.path.join('data',lang+'-test.txt'), 'r')
    max_history = 100
#     history = []
    loss_anything_goes = 0
    loss_from_scratch = 0
    count = 0
    states = None
    total_loss_list = []
    while count<1000:
        c = testfile.read(1)
        if not c:
            break
        
        if count ==0:
            next_char = tf.strings.unicode_split(c, input_encoding='UTF-8')
        else: 
            next_char = tf.strings.unicode_split(c, input_encoding='UTF-8')
        count += 1
        next_char, next_probablity, states = one_step_model.generate_one_step_probablity(next_char, states=states)
        total_loss_list.append(-np.log2(np.max(tf.nn.softmax(next_probablity.numpy()[0]))))
#         if len(history) == max_history:
#             history.pop(0)
#         history.append(c)
    return total_loss_list
        
#         loss_anything_goes -= log2(anything_goes_model(lang, c, history))
#         loss_from_scratch -= log2(from_scratch_model(lang, c, history))
#         if len(history) == max_history:
#             history.pop(0)
#         history.append(c)
#     return [loss_from_scratch/count, loss_anything_goes/count]
     



In [57]:
loss = evaluate_one(lang)

In [58]:
sum(loss)/len(loss)

1.6917308702468872